In [2]:
!pip install DyMat
!pip install scipy
!pip install psycopg2
!pip install bokeh

You should consider upgrading via the '/home/karbo/AAA_PROYECTOS/pv_model/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/karbo/AAA_PROYECTOS/pv_model/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/karbo/AAA_PROYECTOS/pv_model/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/karbo/AAA_PROYECTOS/pv_model/bin/python -m pip install --upgrade pip' command.


# INSERTA DATOS DEL MODELIJUAN AL POSTGRE

In [98]:
import pandas as pd
import numpy as np
import DyMat as DM
from glob import glob
import psycopg2 as pg

import datetime as dt

import matplotlib.pyplot as plt

import bokeh.plotting as bkplt
import bokeh.models as bkmdl
import bokeh.layouts as bklay
import bokeh.palettes as bkpalet #from bokeh.palettes import PuOr
import bokeh.transform as bktrans # from bokeh.transform import cumsum

In [99]:
__MENSAJE_FALLA_LARGO__ = 'LARGO LISTA NO COINCIDE'
__MENSAJE_EXITO__ = 'WENA CONCHETUMARE'

__DATA_RECEIVER_PSQL_LIST_LEN__ = 4
__DATA_TANK_PSQL_LIST_LEN__ = 10
__DATA_POWERBLOCK_PSQL_LIST_LEN__ = 9
__DATA_HELIOSTATS_FIELD_PSQL_LIST_LEN__ = 6 

class CSP_psql_handler:

    def __init__(self,pg_host=None,pg_db=None,pg_usr=None,pg_pwd=None,pg_port=None,pg_conn=None):
        self.pg_host = pg_host
        self.pg_db = pg_db
        self.pg_usr = pg_usr
        self.pg_pwd = pg_pwd
        self.pg_port = pg_port
        
        if ((pg_host != None) and 
            (pg_db != None) and 
            (pg_usr != None) and 
            (pg_pwd != None) and 
            (pg_port != None)):
            self.pg_conn = self.connect_postgre(pg_host,pg_db,pg_usr,pg_pwd,pg_port)
        else:
            self.pg_conn = pg_conn

    def connect_postgre(self,pg_host,pg_db,pg_usr,pg_pwd,pg_port):
        self.pg_conn = pg.connect(host=pg_host,dbname=pg_db,user=pg_usr,password=pg_pwd,port=pg_port)
        return self.pg_conn

    def from_0secs_2dt(self,start_date,tz,seconds):
        dt_obj = dt.datetime.strptime(START_DATE,"%Y-%m-%d") 
        dt_timestamp = dt.datetime.timestamp(dt_obj) 
        dt_obj = dt.datetime.fromtimestamp(seconds+dt_timestamp)+dt.timedelta(hours=tz)
        return dt_obj

    def resample_data(self,df,new_freq_insecs):
        return df.resample(str(new_freq_insecs)+'S').mean()

    def get_periodicity_dbtable(self,of_what,the_what_name,if_tank_is_hot):
        #of_wat: 'tank' | 'receiver' | 'powerblock' | 'heliostatsfield'
        if of_what == 'tank':
            get_period_query = """
                select tank.periodicidad_segundos_tank from tank where tank.nombre_tank = '%s' and tank.hot_tank = %s;
                """%( the_wat_name,('true' if bool(int(if_tank_is_hot)) else 'false') )
        if of_what == 'receiver':
            get_period_query = """
                select receiver.periodicidad_segundos_receiver from receiver where receiver.nombre_receiver = '%s';
                """%(the_what_name)
        if of_what == 'heliostatsfield':
            get_period_query = """
                select heliostat_field.periodicidad_segundos_heliostat_field from heliostat_field where heliostat_field.nombre_heliostat_field = '%s';
                """%(the_what_name)
        if of_what == 'powerblock':
            get_period_query = """
                select power_block.periodicidad_segundos_power_block from power_block where power_block.nombre_power_block = '%s';
                """%(the_what_name)
        
        cur = pg_conn.cursor()        
        try:
            cur.execute(get_period_query)
            freq = fetchall[0]
            pg_conn.commit()
            return freq
        except Exception as e:
            print(e)
            pg_conn.rollback()

    def push_list_one_receiver_medicion(self,list2push):
        
        if len(list2push) == __DATA_RECEIVER_PSQL_LIST_LEN__:
            try:
                cursor = self.pg_conn.cursor()
                cursor.callproc('ins_receiver_medicion',list2push)
                self.pg_conn.commit()
                print(__MENSAJE_EXITO__)
            except Exception as e:
                print(e)
                self.pg_conn.rollback()
        else:
            print(__MENSAJE_FALLA_LARGO__)

    def push_list_one_heliostats_field_medicion(self,list2push):
                
        if len(list2push) == __DATA_HELIOSTATS_FIELD_PSQL_LIST_LEN__:
            try:
                cursor = self.pg_conn.cursor()
                cursor.callproc('ins_heliostat_field_medicion',list2push)
                self.pg_conn.commit()
                print(__MENSAJE_EXITO__)
            except Exception as e:
                print(e)
                self.pg_conn.rollback()
        else:
            print(__MENSAJE_FALLA_LARGO__)

    def push_list_one_tank_medicion(self,list2push):
        
        if len(list2push) == __DATA_POWERBLOCK_PSQL_LIST_LEN__:
            try:
                cursor = self.pg_conn.cursor()
                cursor.callproc('ins_tank_medicion',list2push)
                self.pg_conn.commit()
                print(__MENSAJE_EXITO__)
            except Exception as e:
                print(e)
                self.pg_conn.rollback()
        else:
            print(__MENSAJE_FALLA_LARGO__)

    def push_list_one_tank_medicion(self,list2push):
        
        if len(list2push) == __DATA_TANK_PSQL_LIST_LEN__:
            try:
                cursor = self.pg_conn.cursor()
                cursor.callproc('ins_tank_medicion',list2push)
                self.pg_conn.commit()
                print(__MENSAJE_EXITO__)
            except Exception as e:
                print(e)
                self.pg_conn.rollback()
        else:
            print(__MENSAJE_FALLA_LARGO__)

    def push_df_all_receiver_medicion(self,df,receiver_cols,receiver_name):
        # Itera sobre las filas 
        for index, row in df[receiver_cols].iterrows(): 
            # Create list for the current row 
            rec_list2push = [receiver_name]+[index]+[row[rec_col] for rec_col in receiver_cols]
            print(rec_list2push)
            # append the list to the final list 3
            self.push_list_one_receiver_medicion(rec_list2push)
    
    def push_df_all_powerblock_medicion(self,df,powerblock_cols,powerblock_name):
        # Itera sobre las filas 
        for index, row in df[powerblock_cols].iterrows(): 
            # Create list for the current row 
            pb_list2push = [powerblock_name]+[index]+[row[pb_col] for pb_col in powerblock_cols]
            print(pb_list2push)
            # append the list to the final list 3
            self.push_list_one_receiver_medicion(pb_list2push)
    
    def push_df_all_heliostats_field_medicion(self,df,heliostats_field_cols,heliostats_field_name):
        # Itera sobre las filas 
        for index, row in df[heliostats_field_cols].iterrows(): 
            # Create list for the current row 
            helfield_list2push = [heliostats_field_name]+[index]+[row[helfield_col] for helfield_col in heliostats_field_cols]
            print(helfield_list2push)
            # append the list to the final list 3
            self.push_list_one_heliostats_field_medicion(helfield_list2push)
    
    def push_df_all_tank_medicion(self,df,tank_cols,tank_name,is_hot):
        # Itera sobre las filas 
        for index, row in df[tank_cols].iterrows(): 
            # Create list for the current row 
            tank_list2push = [bool(int(is_hot))]+[tank_name]+[index]+[row[tnk_col] for tnk_col in tank_cols]
            print(tank_list2push)
            # append the list to the final list 3
            self.push_list_one_receiver_medicion(tank_list2push)              

In [100]:
_BASE_COMPONENT_NAME_ = 'ACCIONA_CEDOM'

#ins_powerblock_medicion
_POWERBLOCK_NAME_ = _BASE_COMPONENT_NAME_+str('-')+'PowerBlock'
_POWERBLOCK_COLNAMES_ = ([
    'powerBlock.W_net',
    'powerBlock.E_net',
    'powerBlock.W_gross',
    'powerBlock.fluid_a.m_flow',
    'powerBlock.fluid_a.h_outflow',
    'powerBlock.fluid_a.m_flow',
    'powerBlock.fluid_a.h_outflow',
    ])

_RECEIVER_NAME_ = _BASE_COMPONENT_NAME_+str('-')+'ReceiverTower'
_RECEIVER_COLNAMES_ = ([
    'receiver.heat.Q_flow',
    'receiver.Q_loss',
    ])

_HOT_TANK_1_NAME_ = _BASE_COMPONENT_NAME_+str('-')+'HotTank1'
_HOT_TANK_2_NAME_ = _BASE_COMPONENT_NAME_+str('-')+'HotTank2'
_COLD_TANK_NAME_ = _BASE_COMPONENT_NAME_+str('-')+'ColdTank'

_HOT_TANK_1_COLNAMES_ = ([
    'tankHot.W_loss',
    'tankHot.L',
    'tankHot.Q_losses'
    'tankHot.fluid_a.h_outflow',
    'tankHot.fluid_a.m_flow',
    'tankHot.fluid_b.h_outflow',
    'tankHot.fluid_b.m_flow',
    ])

_HOT_TANK_1_COLNAMES_ = ([
    'tankHot2.W_loss',
    'tankHot2.L',
    'tankHot2.Q_losses'
    'tankHot2.fluid_a.h_outflow',
    'tankHot2.fluid_a.m_flow',
    'tankHot2.fluid_b.h_outflow',
    'tankHot2.fluid_b.m_flow',
    ])

_COLD_TANK_COLNAMES_ = ([
    'tankCold.W_loss',
    'tankCold.L',
    'tankCold.Q_losses'
    'tankCold.fluid_a.h_outflow',
    'tankCold.fluid_a.m_flow',
    'tankCold.fluid_b.h_outflow',
    'tankCold.fluid_b.m_flow',
    ])

_HELIOSTATS_FIELD_NAME_ = _BASE_COMPONENT_NAME_+str('-')+'HeliostatsField'
_HELIOSTATS_FIELD_COLNAMES_ = ([
    'heliostatsField.ele',
    'heliostatsField.azi',
    'heliostatsField.Q_net',
    'heliostatsField.Q_raw'
    ])

In [101]:
matfilename = '/home/karbo/CSP/tower_two_tanks/SolarTherm.SaltTwoTanks/SolarTherm.SaltTwoTanks_res.mat'
#n_res = 3

#d = DM.DyMatFile('../csp_torre_base/csp_torre_res_%d.mat'%(n_res))
d = DM.DyMatFile(matfilename)

var_names1 = list(d.names(1))
var_names1.sort()
var_names2 = list(d.names(2))
var_names2.sort()

#print('block1')
#print(var_names1)
#print('block2')
#print(var_names2)
#print(d.nameTree())
#for el in var_names2:
#    print(str())

In [102]:
data_cols = _HELIOSTATS_FIELD_COLNAMES_

data_arr = d.getVarArray(data_cols)
data_df = pd.DataFrame(data_arr).T
data_df.columns = ['time']+data_cols
data_df = data_df.set_index('time',False)
#print(data_df[data_cols])
#plt.figure(figsize=(50,5))
#plt.plot(data_df['E_elec'])
#plt.figure(figsize=(50,5))
#plt.plot(data_df['P_elec'])

In [103]:
START_DATE = '2020-01-01'
TZ = -3
# object para rellenar el primer argumento
data_df['datetime'] = data_df['time'].apply(lambda x: CSP_psql_handler.from_0secs_2dt(object,START_DATE,TZ,x))
data_df = data_df.set_index('datetime')
data_df = CSP_psql_handler.resample_data(object,data_df,900) # 15 minutos

In [104]:
pg_host = 'sunaios.crykgc3l407t.us-east-2.rds.amazonaws.com'
pg_db = 'sunaios'
pg_usr = 'sunai_sa'
pg_pwd = 'mono1234'
pg_port = '5432'

CSP2db = CSP_psql_handler(
                pg_host = pg_host,
                pg_db = pg_db,
                pg_usr = pg_usr,
                pg_pwd = pg_pwd,
                pg_port = pg_port,
                )


In [91]:
CSP2db.push_df_all_heliostats_field_medicion(data_df,_HELIOSTATS_FIELD_COLNAMES_,_HELIOSTATS_FIELD_NAME_)

['ACCIONA_CEDOM-HeliostatsField', Timestamp('2019-12-31 21:00:00', freq='900S'), 0.0, 2.6425051695921615, 0.0, 0.0]
WENA CONCHETUMARE
['ACCIONA_CEDOM-HeliostatsField', Timestamp('2019-12-31 21:15:00', freq='900S'), 0.0, 2.7120618829901697, 0.0, 0.0]
WENA CONCHETUMARE
['ACCIONA_CEDOM-HeliostatsField', Timestamp('2019-12-31 21:30:00', freq='900S'), 0.0, 2.775466279710277, 0.0, 0.0]
WENA CONCHETUMARE
['ACCIONA_CEDOM-HeliostatsField', Timestamp('2019-12-31 21:45:00', freq='900S'), 0.0, 2.862378465562695, 0.0, 0.0]
WENA CONCHETUMARE
['ACCIONA_CEDOM-HeliostatsField', Timestamp('2019-12-31 22:00:00', freq='900S'), 0.0, 2.931438974572374, 0.0, 0.0]
WENA CONCHETUMARE
['ACCIONA_CEDOM-HeliostatsField', Timestamp('2019-12-31 22:15:00', freq='900S'), 0.0, 3.024336685172726, 0.0, 0.0]
WENA CONCHETUMARE
['ACCIONA_CEDOM-HeliostatsField', Timestamp('2019-12-31 22:30:00', freq='900S'), 0.0, 3.1078035144848193, 0.0, 0.0]
WENA CONCHETUMARE
['ACCIONA_CEDOM-HeliostatsField', Timestamp('2019-12-31 22:45:00',